In [0]:
# Make sure requests in installed
!pip install requests
!pip install kafka-python
!pip install yliveticker

You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-214e0e6b-6934-44ea-b2ab-105813a41117/bin/python -m pip install --upgrade pip' command.
     |████████████████████████████████| 246 kB 7.8 MB/s 
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-214e0e6b-6934-44ea-b2ab-105813a41117/bin/python -m pip install --upgrade pip' command.
     |████████████████████████████████| 55 kB 3.0 MB/s 
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-214e0e6b-6934-44ea-b2ab-105813a41117/bin/python -m pip install --upgrade pip' command.


In [0]:
import requests
import json
from pyspark.sql import SparkSession
import pandas as pd
from sklearn.model_selection import train_test_split
import sklearn
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.sql import SQLContext
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.mllib.evaluation import RegressionMetrics
from pyspark.ml.evaluation import RegressionEvaluator

In [0]:
lark = SparkSession.builder.appName('stocks').getOrCreate()

In [0]:
lark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.0 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
body = {"startDate": "2023-03-20T00:00:39.078Z","endDate": "2023-03-27T20:52:39.078Z"}

In [0]:
# very important to set the header with the bearer token for authentication
my_headers = {"Content-Type" : "application/json", "Bearer" : "ea5afb228d2a4662bf80eb824ac43629"}

In [0]:
response = requests.get("http://ds04-getdata.dnsdojo.org/api/v1/GetKafkaData", headers=my_headers, data=json.dumps(body), timeout=500)
print(f'{response.status_code}: {response.reason}')

200: OK


In [0]:
my_obj = response.json()

In [0]:
#Make df from json from kafka
my_obj = response.json()
df_raw = pd.DataFrame.from_dict(my_obj)

In [0]:
#Show data
df_raw.head()

,ytFinId,tickerID,exchange,quoteType,price,yt_timestamp,marketHours,changePercent,dayVolume,change,priceHint,createdOn
0,56,GM,NYQ,8,34.250000,1679935888000,1,1.601901,5467883,0.540001,2,2023-03-27T12:51:30.3224548
1,57,GM,NYQ,8,34.240002,1679935891000,1,1.572241,5468554,0.530003,2,2023-03-27T12:51:33.2440555
2,58,GM,NYQ,8,34.244999,1679935892000,1,1.587066,5468854,0.535000,2,2023-03-27T12:51:34.2089933
3,59,GM,NYQ,8,34.235001,1679935895000,1,1.557406,5469066,0.525002,2,2023-03-27T12:51:37.3891372
4,60,GM,NYQ,8,34.235001,1679935895000,1,1.557406,5469126,0.525002,2,2023-03-27T12:51:42.2902487


In [0]:
#Set index as ytFinId
df_raw.set_index('ytFinId')

,tickerID,exchange,quoteType,price,yt_timestamp,marketHours,changePercent,dayVolume,change,priceHint,createdOn
ytFinId,,,,,,,,,,,
56,GM,NYQ,8,34.250000,1679935888000,1,1.601901,5467883,0.540001,2,2023-03-27T12:51:30.3224548
57,GM,NYQ,8,34.240002,1679935891000,1,1.572241,5468554,0.530003,2,2023-03-27T12:51:33.2440555
58,GM,NYQ,8,34.244999,1679935892000,1,1.587066,5468854,0.535000,2,2023-03-27T12:51:34.2089933
59,GM,NYQ,8,34.235001,1679935895000,1,1.557406,5469066,0.525002,2,2023-03-27T12:51:37.3891372
60,GM,NYQ,8,34.235001,1679935895000,1,1.557406,5469126,0.525002,2,2023-03-27T12:51:42.2902487
...,...,...,...,...,...,...,...,...,...,...,...
883,GM,NYQ,8,34.411499,1679951289000,2,-0.111761,0,-0.038502,2,2023-03-27T17:08:14.6401953
884,GM,NYQ,8,34.410000,1679951441000,2,-0.116113,0,-0.040001,2,2023-03-27T17:10:45.0515087
885,GM,NYQ,8,34.459999,1679951624000,2,0.029023,0,0.009998,2,2023-03-27T17:13:49.6750551


In [0]:
#Determine which columns are numbers/strings, mean/max/min
df_raw['quoteType'] = df_raw['quoteType'].astype(int)
df_raw.describe()

,ytFinId,quoteType,price,marketHours,changePercent,dayVolume,change,priceHint
count,832.000000,832.0,832.000000,832.000000,832.000000,8.320000e+02,832.000000,832.0
mean,471.500000,8.0,34.364762,1.008413,1.923880,6.211356e+06,0.648537,2.0
std,240.322006,0.0,0.118986,0.091393,0.396814,9.053709e+05,0.133783,0.0
min,56.000000,8.0,34.190800,1.000000,-0.116113,0.000000e+00,-0.040001,2.0
25%,263.750000,8.0,34.259998,1.000000,1.616737,5.650274e+06,0.545002,2.0
50%,471.500000,8.0,34.294998,1.000000,1.735387,5.814794e+06,0.584999,2.0
75%,679.250000,8.0,34.490002,1.000000,2.313861,6.938168e+06,0.780003,2.0
max,887.000000,8.0,34.529999,2.000000,2.432512,7.147106e+06,0.820000,2.0


In [0]:
#Drop NG columns (created on, things that aren't relevant, etc.)
df_useful = df_raw[["price","yt_timestamp","marketHours","changePercent","dayVolume","change"]]

In [0]:
#Make calculated columns
df_useful['yt_timestamp'] = pd.to_datetime(df_useful['yt_timestamp'], unit='ms')
df_useful.head()

<command-1917534184386173>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_useful['yt_timestamp'] = pd.to_datetime(df_useful['yt_timestamp'], unit='ms')


,price,yt_timestamp,marketHours,changePercent,dayVolume,change
0,34.250000,2023-03-27 16:51:28,1,1.601901,5467883,0.540001
1,34.240002,2023-03-27 16:51:31,1,1.572241,5468554,0.530003
2,34.244999,2023-03-27 16:51:32,1,1.587066,5468854,0.535000
3,34.235001,2023-03-27 16:51:35,1,1.557406,5469066,0.525002
4,34.235001,2023-03-27 16:51:35,1,1.557406,5469126,0.525002


In [0]:
grouped_daf = df_useful.resample("min", on='yt_timestamp').mean()
grouped_daf.head()

,price,marketHours,changePercent,dayVolume,change
yt_timestamp,,,,,
2023-03-27 16:51:00,34.235250,1.0,1.558146,5.470726e+06,0.525251
2023-03-27 16:52:00,34.223061,1.0,1.521987,5.481922e+06,0.513062
2023-03-27 16:53:00,34.213396,1.0,1.493316,5.495969e+06,0.503397
2023-03-27 16:54:00,34.239100,1.0,1.569566,5.506386e+06,0.529101
2023-03-27 16:55:00,34.267441,1.0,1.653639,5.539843e+06,0.557442


In [0]:
#Calculate day changes
grouped_daf['yesterChange'] = grouped_daf['price'].diff(1)
grouped_daf['2Change'] = grouped_daf['price'].diff(2)
grouped_daf['3Change'] = grouped_daf['price'].diff(3)
grouped_daf['4Change'] = grouped_daf['price'].diff(4)
grouped_daf['5Change'] = grouped_daf['price'].diff(5)
grouped_daf = grouped_daf.fillna(0)
grouped_daf.head()

,price,marketHours,changePercent,dayVolume,change,yesterChange,2Change,3Change,4Change,5Change
yt_timestamp,,,,,,,,,,
2023-03-27 16:51:00,34.235250,1.0,1.558146,5.470726e+06,0.525251,0.000000,0.000000,0.00000,0.000000,0.0
2023-03-27 16:52:00,34.223061,1.0,1.521987,5.481922e+06,0.513062,-0.012189,0.000000,0.00000,0.000000,0.0
2023-03-27 16:53:00,34.213396,1.0,1.493316,5.495969e+06,0.503397,-0.009665,-0.021854,0.00000,0.000000,0.0
2023-03-27 16:54:00,34.239100,1.0,1.569566,5.506386e+06,0.529101,0.025704,0.016039,0.00385,0.000000,0.0
2023-03-27 16:55:00,34.267441,1.0,1.653639,5.539843e+06,0.557442,0.028341,0.054045,0.04438,0.032191,0.0


In [0]:
#Feature split
nonFeats = ["price"]
feats = [item for item in grouped_daf.columns if item not in nonFeats]

In [0]:
#Convert data
model_data = spark.createDataFrame(grouped_daf)

In [0]:
#Assembler
assy = (VectorAssembler().setInputCols(feats).setOutputCol("Features"))
#assy = VectorIndexer(inputCol = 'Scaled_Feats', outputCol = 'Features')
final_daf = assy.transform(model_data)

In [0]:
#Scaler
scaley = MinMaxScaler(inputCol="Features", outputCol="Scaled_Feats")

In [0]:
#Train test split
training, test = assy.transform(model_data).randomSplit([0.7, 0.3])

In [0]:
#Define model
boosted_model = (GBTRegressor().setLabelCol("price").setFeaturesCol("Scaled_Feats"))

In [0]:
#Pipeline
popeline = Pipeline(stages=[scaley, boosted_model])
the_model = popeline.fit(training)

In [0]:
#Evaluate model on test data
predictions = the_model.transform(test)
score = RegressionEvaluator(predictionCol='prediction', labelCol='price', metricName = 'rmse')
print(score.evaluate(predictions))

0.004650748473955879


In [0]:
#Pull in most recent data (due to current setup will use above dataset, would otherwise make function to repeat whole transformation section)
#For the use case of a prediction program the start and end date could be calculated from those used at the top of the notebook, here they are just incremented manually
#Creating a function to pull in the data and transform it would be more ideal, same process as above
body_predict = {"startDate": "2023-03-28T00:00:39.078Z","endDate": "2023-04-03T20:52:39.078Z"}
#model_data.head()
response_later = requests.get("http://ds04-getdata.dnsdojo.org/api/v1/GetKafkaData", headers=my_headers, data=json.dumps(body), timeout=500)
print(f'{response_later.status_code}: {response_later.reason}')
my_obj = response_later.json()
df_cooked = pd.DataFrame.from_dict(my_obj)
df_cooked.set_index('ytFinId')
df_snoozeful = df_cooked[["price","yt_timestamp","marketHours","changePercent","dayVolume","change"]]
df_snoozeful['yt_timestamp'] = pd.to_datetime(df_snoozeful['yt_timestamp'], unit='ms')

200: OK
<command-1152163904193673>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_snoozeful['yt_timestamp'] = pd.to_datetime(df_snoozeful['yt_timestamp'], unit='ms')


In [0]:
souped_daf = df_snoozeful.resample("min", on='yt_timestamp').mean()

In [0]:
souped_daf.iloc[-10:].tail(15)

,price,marketHours,changePercent,dayVolume,change
yt_timestamp,,,,,
2023-03-27 21:05:00,NaN,NaN,NaN,NaN,NaN
2023-03-27 21:06:00,NaN,NaN,NaN,NaN,NaN
2023-03-27 21:07:00,34.450001,2.0,0.000000,0.0,0.000000
2023-03-27 21:08:00,34.411499,2.0,-0.111761,0.0,-0.038502
2023-03-27 21:09:00,NaN,NaN,NaN,NaN,NaN
2023-03-27 21:10:00,34.410000,2.0,-0.116113,0.0,-0.040001
2023-03-27 21:11:00,NaN,NaN,NaN,NaN,NaN
2023-03-27 21:12:00,NaN,NaN,NaN,NaN,NaN
2023-03-27 21:13:00,34.459999,2.0,0.029023,0.0,0.009998


In [0]:
#Add five rows to end of table, fill hours/change/vol with same as last row
forecasty = souped_daf.iloc[-10:]
forecast = forecasty.append(souped_daf.iloc[-5:].shift(5, freq = 'min'))
#forecast.iloc[-5]
forecast['yesterChange'] = forecast['price'].diff(1)
forecast['2Change'] = forecast['price'].diff(2)
forecast['3Change'] = forecast['price'].diff(3)
forecast['4Change'] = forecast['price'].diff(4)
forecast['5Change'] = forecast['price'].diff(5)
forecast['price'].iloc[-5:] = 0
forecast['dayVolume'].iloc[-5:] = forecast['dayVolume'].iloc[-6]
forecast['change'].iloc[-5:] = forecast['change'].iloc[-6]
forecast['changePercent'].iloc[-5:] = forecast['changePercent'].iloc[-6]
forecast = forecast.fillna(0)
smorecast = forecast.iloc[-5:]
smorecast.tail(1000)

,price,marketHours,changePercent,dayVolume,change,yesterChange,2Change,3Change,4Change,5Change
yt_timestamp,,,,,,,,,,
2023-03-27 21:15:00,0.0,2.0,-0.101602,0.0,-0.035002,-0.004999,-0.049999,0.000000,0.000000,0.0
2023-03-27 21:16:00,0.0,0.0,-0.101602,0.0,-0.035002,0.000000,0.000000,0.000000,0.000000,0.0
2023-03-27 21:17:00,0.0,0.0,-0.101602,0.0,-0.035002,0.000000,0.000000,0.000000,0.000000,0.0
2023-03-27 21:18:00,0.0,2.0,-0.101602,0.0,-0.035002,0.000000,0.000000,0.049999,0.045000,0.0
2023-03-27 21:19:00,0.0,2.0,-0.101602,0.0,-0.035002,-0.045000,0.000000,0.000000,0.004999,0.0


In [0]:
#Format/vectorize new dataset for predictorizing
smodel_data = spark.createDataFrame(smorecast)
new_vector = assy.transform(smodel_data)

In [0]:
#Predict the future
NewPredictions = the_model.transform(new_vector)

In [0]:
#Graph imported data price
NewPredictions = NewPredictions.drop('Features', 'Scaled_Feats')
swoop = NewPredictions.toPandas()
swoop['Time'] = smorecast.index
display(swoop)

price,marketHours,changePercent,dayVolume,change,yesterChange,2Change,3Change,4Change,5Change,prediction,Time
0.0,2.0,-0.10160160809755325,0.0,-0.03500175476074219,-0.0049991607666015625,-0.049999237060546875,0.0,0.0,0.0,34.42585661105102,2023-03-27T21:15:00.000+0000
0.0,0.0,-0.10160160809755325,0.0,-0.03500175476074219,0.0,0.0,0.0,0.0,0.0,-3.3258794794742186E-6,2023-03-27T21:16:00.000+0000
0.0,0.0,-0.10160160809755325,0.0,-0.03500175476074219,0.0,0.0,0.0,0.0,0.0,-3.3258794794742186E-6,2023-03-27T21:17:00.000+0000
0.0,2.0,-0.10160160809755325,0.0,-0.03500175476074219,0.0,0.0,0.049999237060546875,0.04500007629394531,0.0,34.41173090361857,2023-03-27T21:18:00.000+0000
0.0,2.0,-0.10160160809755325,0.0,-0.03500175476074219,-0.04500007629394531,0.0,0.0,0.0049991607666015625,0.0,34.41525345803369,2023-03-27T21:19:00.000+0000


Output can only be rendered in Databricks

In [0]:
#Graph predicted price points
forecast['price'].iloc[-5:] = swoop['prediction']
forecast['Time'] = forecast.index
#forecast.head(15)
display(forecast)

price,marketHours,changePercent,dayVolume,change,yesterChange,2Change,3Change,4Change,5Change,Time
0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2023-03-27T21:05:00.000+0000
0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2023-03-27T21:06:00.000+0000
34.45000076293945,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2023-03-27T21:07:00.000+0000
34.4114990234375,2.0,-0.11176121234893799,0.0,-0.038501739501953125,-0.038501739501953125,0.0,0.0,0.0,0.0,2023-03-27T21:08:00.000+0000
0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2023-03-27T21:09:00.000+0000
34.40999984741211,2.0,-0.11611296236515045,0.0,-0.04000091552734375,0.0,-0.001499176025390625,-0.04000091552734375,0.0,0.0,2023-03-27T21:10:00.000+0000
0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2023-03-27T21:11:00.000+0000
0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2023-03-27T21:12:00.000+0000
34.459999084472656,2.0,0.02902270294725895,0.0,0.009998321533203125,0.0,0.0,0.049999237060546875,0.0,0.04850006103515625,2023-03-27T21:13:00.000+0000
34.41499900817871,2.0,-0.10160160809755325,0.0,-0.03500175476074219,-0.04500007629394531,0.0,0.0,0.0049991607666015625,0.0,2023-03-27T21:14:00.000+0000


Output can only be rendered in Databricks